# Задание

Реализовать классический метод ветвей и границ, в котором нужно ветвиться по дробным переменным для решения Maximum Clique Problem.

# Настройки/Гиперпараметры/Импорты

In [1]:
import numpy as np # для быстрой работы с массивами
import random # для рандомизированного алгоритма
import time # для подсчёта времени работы
import csv # для сохранения ответов
import pandas as pd # для вывода таблицы (v 2.0.0)
import networkx as nx # для поиска независимых множеств (сильные ограничения для BnB в задаче максимальной клики)
import os # для проверки на уже посчитанные решения

from docplex.mp.model import Model # импортирование docplex модели
import cplex # импортирование cplex модели

In [2]:
runs = 1 # число запусков для усреднения времени
iterations = 10000 # сколько попыток делать для поиска хорошего начального решения
random_iters = 75 # число итерация для поиска случайной раскраски графа

recalculate = False # ключ — пересчитывать ли решения для тест-кейсов (если True — решения будут пересчитываться; False — возьмутся из сохранений, если они там есть)

data_path = "./data/" # путь до папки с входными данными

solutions_path = "./solutions/" # путь, куда будут сохраняться посчитанные решения

## Вспомогательные функции, не участвующие в работе алгоритма

In [3]:
def check_solution(edges: dict, solution) -> list:
    """
    Функция для проверка полученного лучшего решения.\n
    Parameters:
        * edges: словарь смежных вершинам вершин, описывающий граф
        * solution: решение в формате [размер клики, [вершины в клике]]\n
    Returns:
        * list — массив с ошибками (пуст, если ошибок нет)
    """
    if solution[0] != len(solution[1]): # проверка соответствия значения целевой функции количеству вершин в клике
        print("Solution objective function does not match variables values")

    errors = [] # массив для ошибок решения

    for i in range(solution[0]-1): # идём по вершинам и проверяем, смежны ли она со всеми остальными вершинами в найденной клике (solution[0] - число вершин в найденной клике)
        for j in range(i+1, solution[0]): # идём по последующим вершинам в клике
            if solution[1][j] not in edges[solution[1][i]]: # проверяем наличие ребра между вершинами
                errors.append([solution[1][i], solution[1][j]]) # если ребра нет, а вершины идут вместе — добавляем ошибку в список ошибок
    return errors # возвращаем ошибки


def transform_solution(solution) -> list:
    """
    Функция для преобразования ответа, чтобы номера вершин шли не с 0, а с 1 и по порядку.\n
    Parameters:
        * solution: решение в формате (размер клики, [вершины в клике])\n
    Returns:
        * list: решение в формате (размер клики, [отсортированные инкрементированные вершины клики])
    """
    for i in range(len(solution[1])): # идём по числу вершин в решении
        solution[1][i] += 1 # инкрементируем номер вершины (чтобы они шли не с 0, а с 1)
    return [solution[0], sorted(solution[1])] # возвращаем решение, попутно отсортировав инкрементированные вершины


def save_solution(dataset, solution: dict) -> None:
    """
    Функция для сохранения лучших ответов.\n
    Parameters:
        * dataset: название тест-кейса
        * solution: словарь для тест-кейса с решениями в формате {"time": время на подсчёт, "clique_size": размер клики, "clique": [вершины, входящие в клику]}\n
    Returns:
        * None: сохраняет решение
    """
    with open(f"{solutions_path}{dataset}.csv", 'w', newline='') as file: # открываем файл для чистой записи
        writer = csv.writer(file) # создаём объект для записи
        writer.writerow([solution["clique_size"]]) # сохраняем размер клики (writerow — сохранение одного элемента в строку)
        writer.writerows([solution["clique"]]) # сохраняем вершины клики (writerows — сохранение итерационных данных по типу списка в строку)
        writer.writerow([solution["time"]]) # сохраняем время работы  (writerow — сохранение одного элемента в строку)


def show_results(solutions: dict) -> None: # solutions - словарь всех полученных ответов
    """
    Функция для вывода таблицы результатов.\n
    Parameters:
        * solutions: словарь решений в формате {"название датасета": {"clique_size": размер клики, "clique": [вершины, входящие в клику], "time": время на подсчёт}, ...}\n
    Returns:
        * None: строит таблицу с полученными ответами
    """
    table = pd.DataFrame(data = [], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"]) # создаём pandas таблицу
    for dataset in solutions.keys(): # идём по тест-кейсам
        testcase = pd.DataFrame(data = [[dataset, solutions[dataset]["time"], solutions[dataset]["clique_size"], solutions[dataset]["clique"]]], columns=["Instance", "Time, sec", "Clique size", "Clique vertices"]) # создаём "новую" таблицу для тест-кейса
        table = pd.concat([table, testcase], ignore_index=True) # объединяем таблицы
    display(table.style.hide()) # скрываем отображение индексов строк таблицы

## Считывание данных

In [4]:
files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "c-fat200-1", "c-fat200-2", "c-fat200-5", "c-fat500-1", "c-fat500-10", "c-fat500-2", "c-fat500-5", "C125.9", "gen200_p0.9_44", "gen200_p0.9_55",  "johnson8-2-4",  "johnson8-4-4", "johnson16-2-4", "hamming6-2", "hamming6-4", "hamming8-2", "hamming8-4", "keller4", "MANN_a9", "MANN_a27", "MANN_a45", "p_hat300-1", "p_hat300-2", "san200_0.7_1", "san200_0.7_2", "san200_0.9_1", "san200_0.9_2", "san200_0.9_3", "sanr200_0.7"] # все файлы, на которых должен быть протестирован код
# skipped = p_hat300-3

# files = ["c-fat200-1", "c-fat200-2", "c-fat200-5", "c-fat500-1", "c-fat500-10", "c-fat500-2", "c-fat500-5", "gen200_p0.9_55",  "johnson8-2-4",  "johnson8-4-4", "johnson16-2-4", "hamming6-2", "hamming6-4", "hamming8-2", "hamming8-4", "MANN_a9", "san200_0.7_1", "san200_0.9_1", "san200_0.9_2"] # лёгкие файлы, на которых должен быть протестирован код
# files = ["brock200_1", "brock200_2", "brock200_3", "brock200_4", "C125.9", "gen200_p0.9_44", "keller4", "MANN_a27", "MANN_a45", "p_hat300-1", "p_hat300-2", "p_hat300-3", "san200_0.7_2", "san200_0.9_3", "sanr200_0.7"] # тяжёлые файлы, на которых должен быть протестирован код

In [5]:
data = {} 
# data - словарь вида 
# {"название датасета" : 
#     {"vertex_num": число вершин, 
#      "edge_num": число рёбер, 
#      "edges": 
#         {словарь вида вершина - set смежных ей вершин}
#     }
#  ...
# }

In [6]:
for file in files:
    data[file] = {"vertex_num": None, "edge_num": None, "edges": {}}
    with open(f"{data_path}{file}.clq", "r") as f: # открываем файл для чтения
        for row in f: # проходим по строкам
            if row[0] == "c": # если строка начинается с буквы "c" - это комментарий, пропускае строку
                continue
            elif row[0] == "p": # если строка начинается с буквы "p" - это описание проблемы, берём из этой строки число вершин и рёбер (последние два числа)
                data[file]["vertex_num"], data[file]["edge_num"] = int(row.split()[-2]), int(row.split()[-1])
            elif row[0] == "e": # если строка начинается с буквы "p" - это вершины, между которыми есть ребро
                v1, v2 = int(row.split()[-2]) - 1, int(row.split()[-1]) - 1 # запоминаем вершины (-1, чтобы не было мороки с индексацией)

                # добавляем связь вершины v1 с v2
                if v1 not in data[file]["edges"].keys(): # если это первое упоминание вершины v1 - создадим для неё set с указанием v2
                    data[file]["edges"][v1] = {v2}
                elif v2 not in data[file]["edges"][v1]: # иначе - просто добавим v2 в set смежных вершин v1
                    data[file]["edges"][v1].add(v2)

                # аналогично, но относительно вершины v2
                if v2 not in data[file]["edges"].keys():
                    data[file]["edges"][v2] = {v1}
                elif v1 not in data[file]["edges"][v2]:
                    data[file]["edges"][v2].add(v1)
        data[file]["edges"] = dict(sorted(data[file]["edges"].items())) # отсортируем вершины в словаре (в set для ключа словаря вершины уже отсортированы)

In [7]:
data["johnson8-2-4"] # пример данных

{'vertex_num': 28,
 'edge_num': 210,
 'edges': {0: {5, 8, 9, 12, 13, 14, 17, 18, 19, 20, 23, 24, 25, 26, 27},
  1: {4, 7, 9, 11, 13, 14, 16, 18, 19, 20, 22, 24, 25, 26, 27},
  2: {3, 6, 9, 10, 13, 14, 15, 18, 19, 20, 21, 24, 25, 26, 27},
  3: {2, 7, 8, 11, 12, 14, 16, 17, 19, 20, 22, 23, 25, 26, 27},
  4: {1, 6, 8, 10, 12, 14, 15, 17, 19, 20, 21, 23, 25, 26, 27},
  5: {0, 6, 7, 10, 11, 14, 15, 16, 19, 20, 21, 22, 25, 26, 27},
  6: {2, 4, 5, 11, 12, 13, 16, 17, 18, 20, 22, 23, 24, 26, 27},
  7: {1, 3, 5, 10, 12, 13, 15, 17, 18, 20, 21, 23, 24, 26, 27},
  8: {0, 3, 4, 10, 11, 13, 15, 16, 18, 20, 21, 22, 24, 26, 27},
  9: {0, 1, 2, 10, 11, 12, 15, 16, 17, 20, 21, 22, 23, 26, 27},
  10: {2, 4, 5, 7, 8, 9, 16, 17, 18, 19, 22, 23, 24, 25, 27},
  11: {1, 3, 5, 6, 8, 9, 15, 17, 18, 19, 21, 23, 24, 25, 27},
  12: {0, 3, 4, 6, 7, 9, 15, 16, 18, 19, 21, 22, 24, 25, 27},
  13: {0, 1, 2, 6, 7, 8, 15, 16, 17, 19, 21, 22, 23, 25, 27},
  14: {0, 1, 2, 3, 4, 5, 15, 16, 17, 18, 21, 22, 23, 24, 27},
  15

# Реализация алгоритма

## Вспомогательные функции, участвующие в работе алгоритма

In [8]:
def randomized_greedy_max_clique(edges:dict, iterations=10) -> list:
    """
    Функция для получения рандомизированного решения задачи о максимальной клике.\n
    Parameters:
        * edges: словарь смежных вершинам вершин
        * iterations: через сколько попыток без улучшения решения выходить из алгоритма\n
    Returns:
        * list: данные о найденной клике в формате [размер найденной клики, [вершина в клике 1, ..., вершина в клике k]]
    """
    original_candidates = set(edges.keys()) # set вершин (изначально все являются кандидатами в клику)
    original_candidates_degrees = [len(edges[v]) for v in original_candidates] # создаём список степеней вершин (индекс - номер вершины, так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)

    attempts = 0 # текущее число попыток
    best_clique = [] # текущая лучшая клика

    while attempts < iterations: # запускаем алгоритм, пока число попыток без изменения результата не превысит счётчик iterations
        clique = [] # создаём "пустую" клику
        candidates = original_candidates.copy() # копируем всех кандидатов
        while len(candidates) != 0: # пока есть кандидаты — пытаемся добавить их в клику
            candidates_degrees = [original_candidates_degrees[i] for i in candidates] # пересчитываем степени кандидатов (оставляем степени только рассматриваемых вершин) для итерациии случайного выбора
            
            v = random.choices(population=list(candidates), weights=candidates_degrees, k=1)[0] # случайным образом выбираем вершину в клику в соответствии с её степенью (чем больше степень относительно других вершин — тем выше вероятность) (переводим candidates в список для случайного выбора)
            clique.append(v) # добавляем её в клику
            
            candidates = candidates.intersection(edges[v]) # среди кандидитов оставляем только тех, кто смежен со всеми вершинами в текущей клике (итеративно этот список постоянно уменьшается с добавлением новых вершин в клику)

        if len(clique) > len(best_clique): # если нашли новую лучшую клику, то запоминаем её
            best_clique = clique.copy()
            attempts = 0 # обнуляем число итераций без улучшения решения
        else:
            attempts += 1 # увеличиваем число итераций без улучшения решения

    return [len(best_clique), best_clique] # возвращаем размер лучшей клики и её саму

In [9]:
def get_independent_sets(edges: dict, random_iters: int=100, min_size_for_ind_set: int=3) -> list:
    """
    Функция для поиска независимых множеств в графе.\n
    Parameters:
        * edges: словарь смежных вершинам вершин
        * random_iters: число итераций для рандомизированного поиска независимых множеств
        * min_size_for_ind_set: минимальный размер для искомых независимых множеств (если равен 1, то сами вершины будут независимыми множествами)\n
    Returns:
        * list: список найденных независимых множеств 
    """
    vertices_num = len(edges) # число вершин в графе
    adj_matrix = np.zeros(shape=(vertices_num, vertices_num), dtype=np.int8) # создание заготовки под матрицу смежности, изначально полностью заполнена нулями

    for v1 in edges.keys(): # идём по вершинам графа
        for v2 in edges[v1]: # идём по смежным v1 вершинам
            adj_matrix[v1][v2] = 1 # отмечаем, что между вершинами есть ребро (из v2 в v1 будет добавлено при рассмотрении вершины v2 как v1)

    G = nx.from_numpy_array(adj_matrix) # создаём объект графа из матрицы смежности

    independent_sets = set() # изначально — сет для tuple-ов, где tuple — независимое множество (в конце конвертируется в список)

    strategies = [nx.coloring.strategy_largest_first, nx.coloring.strategy_independent_set, nx.coloring.strategy_connected_sequential_bfs, nx.coloring.strategy_saturation_largest_first, nx.coloring.strategy_random_sequential] # рассматриваемые стратегии поиска независимых множеств

    for strategy in strategies:
        if strategy == nx.coloring.strategy_random_sequential: # если стратегия — случайное окрашивание
            iters = random_iters # число итераций будет взято из переданных параметров
        else: # иначе — одна итерация, так как другие стратегии всегда будут выдавать один и тот же ответ
            iters = 1

        for _ in range(iters): # запускаем поиск iters раз в зависимости от стратегии 
            coloring_dict = nx.coloring.greedy_color(G, strategy=strategy) # жадно окрашиваем граф по рассматриваемой стратегии
            color2nodes = dict() # словарь для соотнесения цвета с окрашенным им вершинами
            for node, color in coloring_dict.items(): # идём по вершинам и номеру цвета
                if color not in color2nodes.keys(): # если цвета ещё нет в словаре 
                    color2nodes[color] = [] # добавляем под этот цвет пустой массив
                color2nodes[color].append(node) # добавляем в массив соответствующий цвету color вершину, в него окрашенную
            for color, colored_nodes in color2nodes.items(): # идём по цветам и вершинам, окрашенным в этот цвет
                if len(colored_nodes) >= min_size_for_ind_set: # если число вершин цвета color больше заданного порога min_size_for_ind_set
                    colored_nodes = tuple(sorted(colored_nodes)) # сортируем список вершин и конвертируем в tuple (чтобы добавить в set, где не могут дублироваться элементы)
                    independent_sets.add(colored_nodes) # добавление независимого множества (если оно уже было в independent_sets, то оно не добавиться снова)
    independent_sets = [ind_set for ind_set in independent_sets] # конвертируем set в список tuple-ов
    return independent_sets # возвращаем список с независимыми множествами

## Branch and Bound алгоритм

### 1) "Branch and Bound" для задачи о максимальной клике (без использования cplex и ветвления по дробным переменным). Работает быстро, но не соответствует заданию.

In [10]:
class BranchAndBound_DFS():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable) -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения\n
        Returns:
            * None: создаёт модель
        """
        # вершины в алгоритм приходят с нулевой (а не с 1)
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vertices = len(edges) # число вершин в графе
        self.best_clique = [] # текущая лучшая клика
        self.cur_clique = [] # текущая рассматриваемая клика

    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет best_clique
        """
        self.best_clique = self.heuristic_for_init_sol(self.edges, *data) # запоминаем содержимое найденной клики (размер будем брать через len, так как это всё равно за O(1))

    
    def initialize_bnb(self) -> None:
        """
        Функция инициализации Branch and Bound алгоритма для точного поиска максимальной клики.\n
        Returns:
            * None: обновляет best_clique
        """
        #===================================== случайным образом перемешиваем вершины-кандидатов ==========================================
        # candidates = list(range(self.num_vertices)) # изначально кандидаты в клику — все возможные вершины
        # random.shuffle(candidates) # случайно перемешиваем кандидатов
        #------------------------------------- сортируем smallest degree last with remove + reverse ---------------------------------------
        candidates = [-1] * self.num_vertices # массив-заготовка под кандидатов (массив пока заполнен значением -1) (почему-то при создании candidates как numpy.array работа замедляется раз в 5)
        vertices_degrees = [len(self.edges[v]) for v in range(self.num_vertices)] # создаём список степеней вершин (индекс - номер вершины (-1), так как ожидается, что на входе edges остортирован в порядке увеличения номера вершины)
        for i in range(self.num_vertices): # просто итерируемся столько раз, сколько у нас вершин
            vertex = vertices_degrees.index(min(vertices_degrees)) # берём вершину с самой малой степенью
            candidates[i] = vertex # записываем вершину на i-ую позицию с начала (не с конца, так как тут рассматривается reverse версия smallest degree last with remove)
            vertices_degrees[vertex] = np.inf # ставим такую степень найденной вершине, чтобы она больше ни разу не была выбрана
            for v in self.edges[vertex]: # идём по смежным вершинам рассматриваемой
                vertices_degrees[v] -= 1 # понижаем их степени
        #==================================================================================================================================

        self.recursive_bnb(candidates) # запускаем рекурсивный BnB алгоритм (DFS search)
    
    
    def recursive_bnb(self, candidates) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Parameters:
            * candidates: возможные кандидаты для добавления в клику\n
        Returns:
            * None: обновляет best_clique
        """

        if len(candidates) == 0: # если кандидатов не осталось — мы в листе BnB алгоритма
            if len(self.cur_clique) > len(self.best_clique): # если клика в листе получилась лучше — обновляем лучшую
                self.best_clique = self.cur_clique.copy() # запоминаем содержимое клики
            return # выходим из рекурсии, так как кандидатов не осталось
        
        if len(self.cur_clique) + len(candidates) <= len(self.best_clique): # проверка на UB (перестаём рассматривать ветку, если её ответ в любом случае не улучшит максимальную клику)
            return
        
        for i in range(len(candidates)): # идём по возможным кандидатам (ветвление)
            new_candidates = []
            for j in range(i+1, len(candidates)): # идём по оставшимся кандидатам
                # оставляем в новых кандидатах только те вершины, у которых есть ребро с добавляемой вершиной "i" (с вершинами в текущей клике у кандидатов уже проверено наличие ребра)
                if candidates[j] in self.edges[candidates[i]]: # проверяем, есть ли ребро между добавляемым кандидатом i и его "кандидатом по соседству" j
                    new_candidates.append(candidates[j]) # если ребро есть, значит эту вершину j мы можем добавить на следующей итерации алгоритма (j останется в кандидатах на следующем уровне DFS)
            self.cur_clique.append(candidates[i]) # добавляем вершину i в клику
            self.recursive_bnb(new_candidates) # рекурсивно вызываем BnB для оставшихся кандидатов (идём вглубь)
            self.cur_clique.remove(candidates[i]) # убираем добавленную вершину в клику для перехода к следующему (соседнему) элементу в DFS рекурсии
    

    def get_best_solution(self) -> tuple:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Returns:
            * tuple: решение вида (размер клики, [вершины в клике])
        """
        return len(self.best_clique), self.best_clique

#### Тестирование

In [11]:
# solutions_dfs = {} 
# # словарь для ответов алгоритма, основанного на DFS (без использования cplex)
# # {"название датасета" : 
# #     {"clique_size": размер клики,
# #      "clique": [вершины, входящие в клику],
# #      "time": время на подсчёт
# #     }
# # }

In [12]:
# for dataset in data.keys(): # идём по тест-кейсам
#     time_start = time.perf_counter() # замеряем время начала выполнения
#     for i in range(runs): # делаем runs запусков для усреднения времени
#         model_dfs = BranchAndBound_DFS(edges=data[dataset]["edges"], heuristic_for_init_sol=randomized_greedy_max_clique) # создаём объект для модели
#         model_dfs.calc_initial_solution([iterations]) # запускаем поиск начального решения с передачей следующих параметров - (iterations - число запусков рандомизированного алгоритма поиска клики)
#         model_dfs.initialize_bnb() # запускаем Branch and Bound алгоритм
#     time_working = time.perf_counter() - time_start # считаем сколько времени работал алгоритм
#     solution = model_dfs.get_best_solution() # берём полученное решение у модели
#     check_solution(edges=data[dataset]["edges"], solution=solution) # проверка решения
#     solution = transform_solution(solution) # сортирует вершины клики в порядке возрастания их номера и возвращает нумерацию с единицы
#     solutions_dfs[dataset] = {"clique_size": solution[0], "clique": solution[1], "time": time_working/runs} # добавление ответа в словарь с ответами
#     print(f"{dataset}: {solutions_dfs[dataset]}")

In [13]:
# show_results(solutions_dfs)

### 2) Branch and Bound (с использованием docplex). Решает лёгкие задачи и несколько средних, работает медленно даже с максимальной оптимизацией кода.

In [14]:
class BranchAndBound_docplex():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable, check_solution: callable, vars_lb: int=0, vars_ub: int=1, target: str="max") -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения
            * check_solution: функция для проверки правильности решения (должна возвращать ошибки решения в виде массива; если решение правильно — этот список должен быть пуст)
            * vars_lb: минимальное значение, что могут принимать все переменные
            * vars_ub: максимальное значение, что могут принимать все переменные
            * target: на что будет целевая функция, по стандарту — "max"\n
        Returns:
            * None: создаёт модель
        """
        self.model = Model(name="Linear Program") # cplex модель для задачи
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.check_solution = check_solution # функция для проверки правильности решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vertices = len(edges) # число вершин в графе

        self.best_solution = [0, []] # текущее лучшее решение в формате [значение целевой функции, [вершина в клике 1, ..., вершина в клике k]]

        self.vars = [0]*self.num_vertices # список с переменными задачи
        for var_id in range(self.num_vertices): # добавляем в модель непрерывные переменные
            self.vars[var_id] = self.model.continuous_var(name=f"x{var_id}", lb=vars_lb, ub=vars_ub) # добавляем непрерывные переменные-вершины x{var_id}, var_id от 0 до число_вершин_в_графе-1 (-1 из-за нумерации с нуля), что могут принимать значение в диапазоне [vars_lb, vars_ub]

        self.model.set_objective(target, sum(self.vars)) # добавляем целевую функцию в cplex модель (по стандарту — максимизация суммы переменных)


    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет "лучшее" решение
        """
        self.best_solution = self.heuristic_for_init_sol(self.edges, *data) # находим начальное решение эвристикой (запоминаем размер найденной клики и её вершины)
        

    def add_constraints(self, constraints: list, sign: str="<=") -> None:
        """
        Функция для добавления дополнительных ограничений (суммы переменных ? значение) в модель.\n
        Parameters:
            * constraints: список с добавляемыми в модель ограничениями в формате [((номера переменных для суммы в первом ограничении), значение первого ограничения), ..., ((номера переменных для суммы в n-ом ограничении), значение n-го ограничения)]
            * sign: знак добавляемого ограничения, может быть "<=" или ">="\n
        Returns:
            * None: добавляем ограничения в модель
        """
        for constraint in constraints:
            if sign == "<=":
                self.model.add_constraint(sum(self.vars[var] for var in constraint[0]) <= constraint[1]) # добавляем ограничение в виде "сумма переменных <= значение ограничения" (без имени, так как не собираемся его удалять)
            else:
                self.model.add_constraint(sum(self.vars[var] for var in constraint[0]) >= constraint[1]) # добавляем ограничение в виде "сумма переменных >= значение ограничения" (без имени, так как не собираемся его удалять)


    def recursive_bnb(self, eps: float=0.0001) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Parameters:
            * eps: с какой погрешностью считать, что переменная целая\n
        Returns:
            * None: обновляет best_clique
        """

        solution = self.model.solve() # получаем решение задачи от docplex solver-а

        if solution is None: # если получить решение не удалось ввиду (ошибки ограничений и т.д.)
            return # возвращаемся в DFS на уровень выше

        # solution = [round(solution.objective_value), solution.get_all_values()] # преобразовываем solution в формат [int значение целевой функции, [значение переменных в float ~ вершина в клике или нет]]
        solution = [round(solution.objective_value), []] # преобразовываем solution в формат [int значение целевой функции, [пока пустое множество — на случай если решение дробное, чтобы не замедлять алгоритм лишним действием]]

        if solution[0] <= self.best_solution[0]: # проверка на UB (перестаём рассматривать ветку, если она в любом случае не улучшит целевую функцию)
            return # возвращаемся в DFS на уровень выше

        # ищем дробную вершину с наибольшим значением, по которой будем ветвиться (ближайшая к наличию в клике)
        var_fraction = [-1, -1] # дробная переменная в формате [номер переменной, её значение], по которой будем ветвиться (если таковая будет в решении)
        #=========== v1 поиск дробной переменной с наибольшим значением среди всех переменных модели
        # for var_id, var_value in enumerate(solution[1]): # идём по переменным
        #     if not ((1.0 - eps <= var_value <= 1.0 + eps) or (-eps <= var_value <= eps)): # проверяем, дробная ли переменная
        #         if var_value > var_fraction[1]: # если переменная — дробная и её значение самое наибольшее, то будем ветвиться по ней
        #             var_fraction = [var_id, var_value] # запоминаем выбранную переменную
        #----------- v2 поиск дробной переменной с наибольшим значением среди среди только дробных переменных у модели (с помощью iter_continuous_vars)
        for var in self.model.iter_continuous_vars(): # идём по дробным переменным модели
            if not ((1.0 - eps <= var.solution_value <= 1.0 + eps) or (-eps <= var.solution_value <= eps)): # проверяем, дробная ли переменная
                if var.solution_value > var_fraction[1]: # если переменная — дробная и её значение самое наибольшее, то будем ветвиться по ней
                    var_fraction = [var.index, var.solution_value] # запоминаем выбранную переменную
        #===========

        if var_fraction[0] != -1: # если была найдена дробная переменная
            self.vars[var_fraction[0]].set_vartype('B') # переводим переменную в бинарный тип (значения 0 или 1)

            # self.model.add_constraint(self.vars[var_fraction[0]] >= 1, ctname=f"x{var_fraction[0]}>=1") # добавляем ограничение для переменной >=1 (в клике)
            self.model.add_constraint(self.vars[var_fraction[0]] == 1, ctname=f"x{var_fraction[0]}==1") # добавляем ограничение для переменной ==1 (в клике)
            # self.vars[var_fraction[0]].lb = 1 
            self.recursive_bnb() # вызываем BnB на ветку, где дробная переменная >= 1 (с запоминанием текущего ответа)
            # self.model.remove_constraint(f"x{var_fraction[0]}>=1") # убираем добавленное ограничение после того, как DFS пройдёт по его ветке
            self.model.remove_constraint(f"x{var_fraction[0]}==1") # убираем добавленное ограничение после того, как DFS пройдёт по его ветке
            # self.vars[var_fraction[0]].lb = 0 

            # self.model.add_constraint(self.vars[var_fraction[0]] <= 0, ctname=f"x{var_fraction[0]}<=0") # добавляем ограничение для переменной <=0 (не в клике)
            self.model.add_constraint(self.vars[var_fraction[0]] == 0, ctname=f"x{var_fraction[0]}==0") # добавляем ограничение для переменной ==0 (не в клике)
            # self.vars[var_fraction[0]].ub = 0 
            self.recursive_bnb() # вызываем BnB на ветку, где дробная переменная <= 0 (с запоминанием текущего ответа)
            # self.model.remove_constraint(f"x{var_fraction[0]}<=0") # убираем добавленное ограничение после того, как DFS пройдёт по его ветке
            self.model.remove_constraint(f"x{var_fraction[0]}==0") # убираем добавленное ограничение после того, как DFS пройдёт по его ветке
            # self.vars[var_fraction[0]].ub = 1

            self.vars[var_fraction[0]].set_vartype('C') # переводим переменную в непрерывный тип (значения от lb до ub, что были заданы ранее)
        #===================== v1 с проверкой решения прямо в модели (если не были переданы слабые ограничения)
        else: # если дробных переменных нет — проверяем решение на корректность
            # solution_ = [solution[0], [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)]] # в solution_ заменяем переменные на вершины, что "находятся в клике"
            # в [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)] пронумеровываем все вершины в решении и берём номера только тех вершин, у которых значение = 1 (состоят в клике)
            solution_ = [solution[0], [var.index for var in self.model.iter_variables() if (1.0 - eps <= var.solution_value <= 1.0 + eps)]] # в solution_ заменяем переменные на вершины, что "находятся в клике"

            errors = self.check_solution(self.edges, solution_) # получаем список ошибок решения (если решение без ошибок, то check_solution должен вернуть пустой список)
            if len(errors) != 0: # если в решении есть ошибки (если ошибок не было — список будет пустым)
                for error in errors: # идём по парам ошибочно связанных вершин (error имеет формат [переменная 1, переменная 2]; если ошибок не было — список будет пустым)
                    self.model.add_constraint(self.vars[error[0]] + self.vars[error[1]] <= 1.0, ctname=f"x{error[0]}+x{error[1]}<=1") # добавляем ограничение "xi + xj <= 1"
                self.recursive_bnb() # вызываем BnB для пересчёта текущего решения
            elif solution_[0] > self.best_solution[0]: # если значение целевой функции улучшилось (все переменные не дробные и решение корректное)
                self.best_solution = solution_.copy() # сохраняем полученное лучшее решение
        #--------------------- v2 без проверки решения, если у модели имеются все необходимые данные (были переданы слабые ограничения)
        # elif solution[0] > self.best_solution[0]: # если значение целевой функции улучшилось (все переменные не дробные и решение корректное)
        #     self.best_solution = solution.copy() # сохраняем полученное лучшее решение
        #=====================
        return # выходим из рекурсии (возвращаемся на уровень выше в DFS)
        

    def get_best_solution(self, eps: float=0.0001) -> list:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Parameters:
            * eps: с какой погрешностью считать, что переменная целая\n
        Returns:
            * list: решение вида [значение целевой функции, [вершина в клике 1, ..., вершина в клике k]]
        """
        for v in self.best_solution[1]: # идём по переменным (либо номерам вершин в клике)
            if isinstance(v, float): # если в ответе есть не целые числа ~ переменные (ещё не сконвертированные в номера вершин)
                self.best_solution[1] = [var_id for var_id, var_value in enumerate(self.best_solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)] # в best_solution заменяем переменные на вершины, что "находятся в клике"
                break # выходим из цикла
        return self.best_solution

### 3) Branch and Bound (с использованием cplex)

In [15]:
class BranchAndBound():
    def __init__(self, edges: dict, heuristic_for_init_sol: callable, check_solution: callable, vars_lb: float=0.0, vars_ub: float=1.0, target: str="max", eps: float=0.0001) -> None: # инициализация модели для решения задачи
        """
        Конструктор для модели.\n
        Parameters:
            * edges: словарь смежных вершинам вершин
            * heuristic_for_init_sol: функция, что будет использоваться для нахождения первичного решения
            * check_solution: функция для проверки правильности решения (должна возвращать ошибки решения в виде массива; если решение правильно — этот список должен быть пуст)
            * vars_lb: минимальное значение, что могут принимать все переменные
            * vars_ub: максимальное значение, что могут принимать все переменные
            * target: на что будет целевая функция ("max" или "min"), по стандарту — "max"
            * eps: с какой погрешностью считать, что переменная целая\n
        Returns:
            * None: создаёт модель
        """
        self.heuristic_for_init_sol = heuristic_for_init_sol # эвристическая функция для поиска начального решения
        self.check_solution = check_solution # функция для проверки правильности решения
        self.edges = edges # данные о изначальном графе (словарь смежных вершинам вершин)
        self.num_vars = len(edges) # число переменных в задаче (вершин в графе)
        self.eps = eps # с какой погрешностью считать, что переменная целая
        self.best_solution = [0, []] # текущее лучшее решение в формате [значение целевой функции, [вершина в клике 1, ..., вершина в клике k]]

        self.model = cplex.Cplex() # создание объекта для модели

        obj = [1.0] * self.num_vars # коэффициенты переменных для целевой функции (их количество равно числу переменных)
        lb = [vars_lb] * self.num_vars # lower bound-ы для переменных модели (их количество равно числу переменных)
        ub = [vars_ub] * self.num_vars # upper bound-ы для переменных модели (их количество равно числу переменных)
        names = [f"x{i}" for i in range(self.num_vars )] # имена для переменных модели (их количество равно числу переменных)
        types = ["C"] * self.num_vars # типы для переменных модели ("C" - непрерывная, "B" — бинарная, "I" — целочисленная) (их количество равно числу переменных)
        self.model.variables.add(obj=obj, lb=lb, ub=ub, names=names, types=types) # добавление переменных в модель

        self.model.objective.set_name("Linear Program") # название модели (опционально)
        if target == "max": # тип целевой функции (max или min)
            self.model.objective.set_sense(self.model.objective.sense.maximize) # тип целевой функции — максимизация
        else:
            self.model.objective.set_sense(self.model.objective.sense.minimize) # тип целевой функции — минимизация

        self.model.set_log_stream(None) # отключение логирования у модели
        self.model.set_error_stream(None) # отключение оповещений об ошибках у модели (они handle-ятся иначе)
        self.model.set_warning_stream(None) # отключение предупреждений у модели (они handle-ятся иначе)
        self.model.set_results_stream(None) # отключение оповещений о результатах решения у модели 


    def calc_initial_solution(self, data) -> None:
        """
        Функция для поиска начального решения.\n
        Parameters:
            * data: дополнительные параметры, что будут отправлены в функцию поиска начального решения\n
        Returns:
            * None: обновляет "лучшее" решение
        """
        self.best_solution = self.heuristic_for_init_sol(self.edges, *data) # находим начальное решение эвристикой (запоминаем размер найденной клики и её вершины)
        

    def add_constraints(self, constraints: list, senses: list, rhs: list) -> None:
        """
        Функция для добавления дополнительных ограничений (суммы переменных ? значение) в модель.\n
        Parameters:
            * constraints: список с добавляемыми в модель ограничениями в формате [[[номера/имена переменных в первом ограничении], [коэффициенты для переменных в первом ограничении]], ..., [[номера/имена переменных в n-ом ограничении], [коэффициенты для переменных в n-ом ограничении]]]
            * senses: список со знаками добавляемых ограничений, может быть "G"~">="   "L"~"<="   "E"~"==" в формате [знак ограничения 1, ..., знак ограничения n]
            * rhs: список с границами (правыми частями) для ограничений в формате [ограничение 1, ..., ограничение n]\n
        Returns:
            * None: добавляем ограничения в модель
        """
        self.model.linear_constraints.add(lin_expr=constraints, senses=senses, rhs=rhs) # добавляем ограничения (без имён) в модель


    def recursive_bnb(self) -> None:
        """
        Функция для запуска точного алгоритма Branch and Bound.\n
        Returns:
            * None: обновляет best_solution
        """
        try: # пытаемся получить решение
            self.model.solve() # считаем решение от cplex solver-а
            solution = [int(self.model.solution.get_objective_value()+0.01), self.model.solution.get_values()] # создаём solution в формате [int значение целевой функции +0.01 на случай дробного решения чуть меньше целого числа, [значение переменных модели]]
        except cplex.exceptions.CplexSolverError as error: # если получили ошибку solver-а (например — неправильно наложившиеся ограничения из-за которых задача не решается)
            return # возвращаемся в DFS на уровень выше
        
        if solution[0] <= self.best_solution[0]: # проверка на UB (перестаём рассматривать ветку, если она в любом случае не улучшит целевую функцию)
            return # возвращаемся в DFS на уровень выше

        # ищем дробную вершину с наибольшим значением, по которой будем ветвиться (ближайшая к наличию в клике)
        var_fraction = [-1, -1] # дробная переменная в формате [номер переменной, её значение], по которой будем ветвиться (если таковая будет в решении)
        for var_id, var_value in enumerate(solution[1]): # идём по переменным в ответе
            if not ((1.0 - self.eps <= var_value <= 1.0 + self.eps) or (-self.eps <= var_value <= self.eps)): # проверяем, дробная ли переменная
                if var_value > var_fraction[1]: # если переменная — дробная и её значение самое наибольшее, то будем ветвиться по ней
                    var_fraction = [var_id, var_value] # запоминаем выбранную переменную


        if var_fraction[0] != -1: # если была найдена дробная переменная
            self.model.variables.set_types(var_fraction[0], "B") # переводим переменную в бинарный тип (значения 0 или 1)
            
            self.model.linear_constraints.add(lin_expr=[[[var_fraction[0]], [1.0]]], senses=["E"], rhs=[1.0], names=[f"x{var_fraction[0]}==1"]) # добавляем ограничение для переменной ==1 (в клике)
            self.recursive_bnb() # вызываем BnB на ветку, где дробная переменная >= 1 (с запоминанием текущего ответа)
            self.model.linear_constraints.delete(f"x{var_fraction[0]}==1") # убираем добавленное ограничение по имени после того, как DFS пройдёт по его ветке

            self.model.linear_constraints.add(lin_expr=[[[var_fraction[0]], [1.0]]], senses=["E"], rhs=[0.0], names=[f"x{var_fraction[0]}==0"]) # добавляем ограничение для переменной ==0 (не в клике)
            self.recursive_bnb() # вызываем BnB на ветку, где дробная переменная <= 0 (с запоминанием текущего ответа)
            self.model.linear_constraints.delete(f"x{var_fraction[0]}==0") # убираем добавленное ограничение по имени после того, как DFS пройдёт по его ветке

            self.model.variables.set_types(var_fraction[0], "C") # переводим переменную в непрерывный тип (значения от lb до ub, что были заданы ранее)
        #===================== v1 с проверкой решения прямо в модели (если не были переданы слабые ограничения)
        else: # если дробных переменных нет — проверяем решение на корректность
            solution = [solution[0], [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - self.eps <= var_value <= 1.0 + self.eps)]] # в solution заменяем переменные на вершины, что "находятся в клике"
            # в [var_id for var_id, var_value in enumerate(solution[1]) if (1.0 - self.eps <= var_value <= 1.0 + self.eps)] пронумеровываем все вершины в решении и берём номера только тех вершин, у которых значение = 1 (состоят в клике)

            errors = self.check_solution(self.edges, solution) # получаем список ошибок решения (если решение без ошибок, то check_solution должен вернуть пустой список)
            if len(errors) != 0: # если в решении есть ошибки (если ошибок не было — список будет пустым)
                constraints = list(zip(errors, [[1.0, 1.0] for i in range(len(errors))])) # конвертируем список пар ошибочно связанных вершин xi и xj в формат [([xi, xj], [1.0, 1.0]), ...], где [1.0, 1.0] — веса переменных в ограничениях
                senses = ["L"] * len(errors) # тип ограничения "L"~"<=" для всех найденных ошибок
                rhs = [1.0] * len(errors) # значение ограничений для ошибок
                self.add_constraints(constraints=constraints, senses=senses, rhs=rhs) # добавляем ограничение "xi + xj <= 1"

                self.recursive_bnb() # вызываем BnB для пересчёта текущего решения
            elif solution[0] > self.best_solution[0]: # если значение целевой функции улучшилось (все переменные не дробные и решение корректное)
                self.best_solution = solution.copy() # сохраняем полученное лучшее решение
        #--------------------- v2 без проверки решения, если у модели имеются все необходимые данные (были переданы слабые ограничения)
        # elif solution[0] > self.best_solution[0]: # если значение целевой функции улучшилось (все переменные не дробные и решение корректное)
        #     self.best_solution = solution.copy() # сохраняем полученное лучшее решение
        #=====================
        return # выходим из рекурсии (возвращаемся на уровень выше в DFS)
        

    def get_best_solution(self, eps: float=0.0001) -> list:
        """
        Функция, возвращающая лучшее найденное решение.\n
        Parameters:
            * eps: с какой погрешностью считать, что переменная целая\n
        Returns:
            * list: решение вида [значение целевой функции, [вершина в клике 1, ..., вершина в клике k]]
        """
        for v in self.best_solution[1]: # идём по переменным (либо номерам вершин в клике)
            if isinstance(v, float): # если в ответе есть не целые числа ~ переменные (ещё не сконвертированные в номера вершин)
                self.best_solution[1] = [var_id for var_id, var_value in enumerate(self.best_solution[1]) if (1.0 - eps <= var_value <= 1.0 + eps)] # в best_solution заменяем переменные на вершины, что "находятся в клике"
                break # выходим из цикла
        return self.best_solution

#### Тестирование

In [16]:
solutions_cplex = {} 
# словарь для ответов алгоритма (соответствующего заданию)
# {"название датасета" : 
#     {"time": время на подсчёт,
#      "clique_size": размер клики,
#      "clique": [вершины, входящие в клику]
#     }
# }

In [17]:
for dataset in data.keys(): # идём по тест-кейсам
    if os.path.exists(f"{solutions_path}{dataset}.csv") and not recalculate: # если ответ для тест-кейса уже был посчитан ранее — загружаем его
        with open(f"{solutions_path}{dataset}.csv", 'r') as file: # открываем файл для чтения (не бинарного)
            reader = csv.reader(file) # создаём объект для чтения
            reader = list(reader) # конвертируем объект _csv.reader в список для лёгкой итерации
            solutions_cplex[dataset] = {} # создаём пустой словарь под тест-кейс
            solutions_cplex[dataset]["time"] = float(reader[2][0]) # конвертируем элемент третьей строки (время работы алгоритма) в float
            solutions_cplex[dataset]["clique_size"] = int(reader[0][0]) # конвертируем элемент первой строки (размер клики) в int
            solutions_cplex[dataset]["clique"] = [int(node) for node in reader[1]] # конвертируем вторую строку (с вершинами клики) в массив int-ов
    else: # если кейс ещё не решён — решаем
        edges = data[dataset]["edges"] # смежные вершины в тест-кейсе
        time_start = time.perf_counter() # замеряем время начала выполнения
        for i in range(runs): # делаем runs запусков для усреднения времени
            model_cplex = BranchAndBound(edges=edges, heuristic_for_init_sol=randomized_greedy_max_clique, check_solution=check_solution) # создаём BnB модель
            model_cplex.calc_initial_solution([iterations]) # запускаем поиск начального решения с передачей следующих параметров - (iterations - число запусков рандомизированного алгоритма поиска клики)
            print(f"{dataset} solution from heuristic: {model_cplex.get_best_solution()}") # вывод начального эвристического решения
            
        
            independent_sets = get_independent_sets(edges=edges, random_iters=random_iters) # считаем независимые множества в графе (без них модель будет работать медленно на слабых ограничениях)
            for i in range(len(independent_sets)): # идём по независимым множествам и конвертируем их в данные для неравенств
                independent_sets[i] = (independent_sets[i], [1.0]*len(independent_sets[i])) # ((номера переменных), [коэффициенты переменных для ограничений])
            senses = ["L"] * len(independent_sets) # тип ограничения "L"~"<=" для всех найденных независимых множеств
            rhs = [1.0] * len(independent_sets) # значение ограничений для ошибок
            model_cplex.add_constraints(constraints=independent_sets, senses=senses, rhs=rhs) # добавляем сильные ограничения в модель


            #===================== v1 с проверкой решения прямо в модели (если не были переданы слабые ограничения)

            #--------------------- v2 без проверки решения ~ передаём в модель все необходимые данные (слабые ограничения)
            # simple_restrictions = check_solution(edges=edges, solution=(len(edges.keys()), list(range(len(edges.keys()))))) # получаем все слабые ограничения для графа с помощью фиктивного решения из всех элементов этого графа
            # simple_restrictions = list(zip(simple_restrictions, [[1.0, 1.0] for i in range(len(simple_restrictions))])) # конвертируем список пар ошибочно связанных вершин xi и xj в формат [([xi, xj], [1.0, 1.0]), ...], где [1.0, 1.0] — веса переменных в ограничениях
            # senses = ["L"] * len(simple_restrictions) # тип ограничения "L"~"<=" для всех найденных ошибок
            # rhs = [1.0] * len(simple_restrictions) # значение ограничений для ошибок
            # model_cplex.add_constraints(constraints=simple_restrictions, senses=senses, rhs=rhs) # добавляем ограничение "xi + xj <= 1"
            #=====================

            model_cplex.recursive_bnb() # запускаем рекурсивный BnB алгоритм
        time_working = time.perf_counter() - time_start # считаем сколько времени работал алгоритм

        solution = model_cplex.get_best_solution() # получаем точный ответ алгоритма (на ошибки он уже проверен в ходе выполнения BnB)
        if len(check_solution(edges=edges, solution=solution)) != 0: # проверка на то, что в решении не нашлось ошибок (check_solution возвращает список с ошибками, если их нет — список пустой)
            raise Exception("Solution is incorrect") # выбрасываем исключение
        solution = transform_solution(solution) # сортирует вершины клики в порядке возрастания их номера и возвращает нумерацию с единицы
        solutions_cplex[dataset] = {"time": time_working/runs, "clique_size": solution[0], "clique": solution[1]} # добавление ответа в словарь с ответами
        save_solution(dataset=dataset, solution=solutions_cplex[dataset]) # сохраняем решение для рассматриваемого кейса
    print(f"{dataset}: {solutions_cplex[dataset]}")

brock200_1: {'clique_size': 21, 'clique': [4, 26, 32, 41, 46, 48, 83, 100, 103, 104, 107, 120, 122, 132, 137, 138, 144, 175, 180, 191, 199], 'time': 5591.3516146}
brock200_2: {'clique_size': 12, 'clique': [27, 48, 55, 70, 105, 120, 121, 135, 145, 149, 158, 183], 'time': 143.21986219999962}
brock200_3: {'clique_size': 15, 'clique': [12, 29, 36, 38, 58, 84, 97, 98, 104, 118, 130, 144, 158, 173, 178], 'time': 668.4389148}
brock200_4: {'clique_size': 17, 'clique': [12, 19, 28, 29, 38, 54, 65, 71, 79, 93, 117, 127, 139, 161, 165, 186, 192], 'time': 1257.8381762999998}
c-fat200-1: {'clique_size': 12, 'clique': [7, 8, 44, 45, 81, 82, 118, 119, 155, 156, 192, 193], 'time': 1.6992627999998149}
c-fat200-2: {'clique_size': 24, 'clique': [1, 2, 19, 20, 37, 38, 55, 56, 73, 74, 91, 92, 109, 110, 127, 128, 145, 146, 163, 164, 181, 182, 199, 200], 'time': 2.851880899999742}
c-fat200-5: {'clique_size': 58, 'clique': [3, 4, 10, 11, 17, 18, 24, 25, 31, 32, 38, 39, 45, 46, 52, 53, 59, 60, 66, 67, 73, 74, 

In [18]:
show_results(solutions_cplex) # вывод таблицы с результатами

Instance,"Time, sec",Clique size,Clique vertices
brock200_1,5591.351615,21,"[4, 26, 32, 41, 46, 48, 83, 100, 103, 104, 107, 120, 122, 132, 137, 138, 144, 175, 180, 191, 199]"
brock200_2,143.219862,12,"[27, 48, 55, 70, 105, 120, 121, 135, 145, 149, 158, 183]"
brock200_3,668.438915,15,"[12, 29, 36, 38, 58, 84, 97, 98, 104, 118, 130, 144, 158, 173, 178]"
brock200_4,1257.838176,17,"[12, 19, 28, 29, 38, 54, 65, 71, 79, 93, 117, 127, 139, 161, 165, 186, 192]"
c-fat200-1,1.699263,12,"[7, 8, 44, 45, 81, 82, 118, 119, 155, 156, 192, 193]"
c-fat200-2,2.851881,24,"[1, 2, 19, 20, 37, 38, 55, 56, 73, 74, 91, 92, 109, 110, 127, 128, 145, 146, 163, 164, 181, 182, 199, 200]"
c-fat200-5,11.434945,58,"[3, 4, 10, 11, 17, 18, 24, 25, 31, 32, 38, 39, 45, 46, 52, 53, 59, 60, 66, 67, 73, 74, 80, 81, 87, 88, 94, 95, 101, 102, 108, 109, 115, 116, 122, 123, 129, 130, 136, 137, 143, 144, 150, 151, 157, 158, 164, 165, 171, 172, 178, 179, 185, 186, 192, 193, 199, 200]"
c-fat500-1,5.822595,14,"[9, 10, 89, 90, 169, 170, 249, 250, 329, 330, 409, 410, 489, 490]"
c-fat500-10,43.546177,126,"[3, 4, 11, 12, 19, 20, 27, 28, 35, 36, 43, 44, 51, 52, 59, 60, 67, 68, 75, 76, 83, 84, 91, 92, 99, 100, 107, 108, 115, 116, 123, 124, 131, 132, 139, 140, 147, 148, 155, 156, 163, 164, 171, 172, 179, 180, 187, 188, 195, 196, 203, 204, 211, 212, 219, 220, 227, 228, 235, 236, 243, 244, 251, 252, 259, 260, 267, 268, 275, 276, 283, 284, 291, 292, 299, 300, 307, 308, 315, 316, 323, 324, 331, 332, 339, 340, 347, 348, 355, 356, 363, 364, 371, 372, 379, 380, 387, 388, 395, 396, 403, 404, 411, 412, 419, 420, 427, 428, 435, 436, 443, 444, 451, 452, 459, 460, 467, 468, 475, 476, 483, 484, 491, 492, 499, 500]"
c-fat500-2,7.654373,26,"[9, 10, 49, 50, 89, 90, 129, 130, 169, 170, 209, 210, 249, 250, 289, 290, 329, 330, 369, 370, 409, 410, 449, 450, 489, 490]"
